In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import json
import pandas as pd
import numpy as np
import sys

pd.set_option('display.max_colwidth', None)
sys.path.append('./src-py')

In [4]:
%autoreload
import sbert_training
from sklearn.metrics import precision_recall_fscore_support

In [5]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import TripletEvaluator
from datetime import datetime
from sentence_transformers import util
from zipfile import ZipFile
from sentence_transformers.datasets import SentenceLabelDataset
from sentence_transformers.datasets import NoDuplicatesDataLoader

import logging

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)

In [6]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taskb_training_df = pd.read_csv('../data/TaskB_train.csv')

taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')
taskb_valid_df = pd.read_csv('../data/TaskB_dev.csv')

In [7]:
output_path = "../../data-ceph/arguana/argmining22-sharedtask/models/"

In [8]:
#Mapping labels
taska_training_df = taska_training_df[taska_training_df.Validity != 0]
taska_valid_df    = taska_valid_df[taska_valid_df.Validity != 0]

taska_training_df['label'] = taska_training_df.Validity.apply(lambda x : 1 if x == 1 else 0)
taska_valid_df['label'] = taska_valid_df.Validity.apply(lambda x : 1 if x == 1 else 0)

taska_training_df['Premise'] = taska_training_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)
taska_valid_df['Premise'] = taska_valid_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)

In [9]:
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '[CLS] {} [SEP] {} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)

In [10]:
taska_training_df.head()

,topic,Premise,Conclusion,Validity,Validity-Confidence,Novelty,Novelty-Confidence,label,input_txt
0,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Depression is a well-known psychological problem of modern society that is partly caused by TV watching:,1,confident,1,confident,1,"[CLS] TV viewing is harmful to children [SEP] TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions. [SEP] Depression is a well-known psychological problem of modern society that is partly caused by TV watching: [SEP]"
1,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Children's TV viewing fosters negative emotions,1,very confident,-1,majority,1,"[CLS] TV viewing is harmful to children [SEP] TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions. [SEP] Children's TV viewing fosters negative emotions [SEP]"
2,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Popularity of TV is harmful to children,1,very confident,1,majority,1,"[CLS] TV viewing is harmful to children [SEP] TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions. [SEP] Popularity of TV is harmful to children [SEP]"
3,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Violence on TV and in movies encourages psychological stress,1,very confident,1,majority,1,"[CLS] TV viewing is harmful to children [SEP] TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions. [SEP] Violence on TV and in movies encourages psychological stress [SEP]"
4,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",US-India deal does not cap or limit Indian fissile material production.,-1,very confident,-1,very confident,0,"[CLS] TV viewing is harmful to children [SEP] TV viewing is har

In [11]:
taska_training_df.label.value_counts()

1    401
0    320
Name: label, dtype: int64

In [12]:
taska_valid_df.label.value_counts()

1    125
0     74
Name: label, dtype: int64

In [14]:
#train_model(taska_training_df, taska_valid_df, '../data/output/', 'sentence-transformers/nli-roberta-large', num_epochs=20, train_batch_size=16, model_suffix='', data_file_suffix='', max_seq_length=256, special_tokens=[], loss='ContrastiveLoss', sentence_transformer=False)

In [13]:
_, _ = sbert_training.train_model(taska_training_df, taska_valid_df, 
                                    output_path + '/task-A/validity/sbert/', 
                                    'sentence-transformers/nli-roberta-large', 
                                    num_epochs=15, train_batch_size=32,
                                    model_suffix='', max_seq_length=512, special_tokens=[], 
                                    loss='ContrastiveLoss', sentence_transformer=False, evaluation_steps=10)

2022-07-02 19:06:24 - Use pytorch device: cuda
2022-07-02 19:06:24 - Read Triplet train dataset
Len of training: 721
Len of Dev: 199
Evaluating before start learning.....
2022-07-02 19:06:24 - Binary Accuracy Evaluation of the model on sts-dev dataset:
2022-07-02 19:06:27 - Accuracy with Cosine-Similarity:           72.36	(Threshold: 0.6503)
2022-07-02 19:06:27 - F1 with Cosine-Similarity:                 80.69	(Threshold: 0.5985)
2022-07-02 19:06:27 - Precision with Cosine-Similarity:          70.91
2022-07-02 19:06:27 - Recall with Cosine-Similarity:             93.60
2022-07-02 19:06:27 - Average Precision with Cosine-Similarity:  82.54

2022-07-02 19:06:27 - Accuracy with Manhatten-Distance:           72.36	(Threshold: 646.3732)
2022-07-02 19:06:27 - F1 with Manhatten-Distance:                 80.56	(Threshold: 679.7071)
2022-07-02 19:06:27 - Precision with Manhatten-Distance:          71.17
2022-07-02 19:06:27 - Recall with Manhatten-Distance:             92.80
2022-07-02 19:06:27

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

2022-07-02 19:06:30 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 0 after 10 steps:
2022-07-02 19:06:30 - Accuracy with Cosine-Similarity:           72.36	(Threshold: 0.6616)
2022-07-02 19:06:30 - F1 with Cosine-Similarity:                 80.41	(Threshold: 0.6062)
2022-07-02 19:06:30 - Precision with Cosine-Similarity:          70.48
2022-07-02 19:06:30 - Recall with Cosine-Similarity:             93.60
2022-07-02 19:06:30 - Average Precision with Cosine-Similarity:  82.40

2022-07-02 19:06:30 - Accuracy with Manhatten-Distance:           72.36	(Threshold: 631.9717)
2022-07-02 19:06:30 - F1 with Manhatten-Distance:                 80.56	(Threshold: 672.8953)
2022-07-02 19:06:30 - Precision with Manhatten-Distance:          71.17
2022-07-02 19:06:30 - Recall with Manhatten-Distance:             92.80
2022-07-02 19:06:30 - Average Precision with Manhatten-Distance:  82.55

2022-07-02 19:06:30 - Accuracy with Euclidean-Distance:           72.36	(Threshold: 25.3380

...

2022-07-02 19:08:01 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 12 after 10 steps:
2022-07-02 19:08:01 - Accuracy with Cosine-Similarity:           73.87	(Threshold: 0.7212)
2022-07-02 19:08:01 - F1 with Cosine-Similarity:                 81.94	(Threshold: 0.7212)
2022-07-02 19:08:01 - Precision with Cosine-Similarity:          72.39
2022-07-02 19:08:01 - Recall with Cosine-Similarity:             94.40
2022-07-02 19:08:01 - Average Precision with Cosine-Similarity:  81.21

2022-07-02 19:08:01 - Accuracy with Manhatten-Distance:           73.87	(Threshold: 575.0896)
2022-07-02 19:08:01 - F1 with Manhatten-Distance:                 82.07	(Threshold: 575.0896)
2022-07-02 19:08:01 - Precision with Manhatten-Distance:          72.12
2022-07-02 19:08:01 - Recall with Manhatten-Distance:             95.20
2022-07-02 19:08:01 - Average Precision with Manhatten-Distance:  81.42

2022-07-02 19:08:01 - Accuracy with Euclidean-Distance:           73.87	(Threshold: 23.073

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

2022-07-02 19:08:06 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 13 after 10 steps:
2022-07-02 19:08:06 - Accuracy with Cosine-Similarity:           73.87	(Threshold: 0.7220)
2022-07-02 19:08:06 - F1 with Cosine-Similarity:                 81.94	(Threshold: 0.7220)
2022-07-02 19:08:06 - Precision with Cosine-Similarity:          72.39
2022-07-02 19:08:06 - Recall with Cosine-Similarity:             94.40
2022-07-02 19:08:06 - Average Precision with Cosine-Similarity:  81.10

2022-07-02 19:08:06 - Accuracy with Manhatten-Distance:           73.87	(Threshold: 574.9849)
2022-07-02 19:08:06 - F1 with Manhatten-Distance:                 82.07	(Threshold: 574.9849)
2022-07-02 19:08:06 - Precision with Manhatten-Distance:          72.12
2022-07-02 19:08:06 - Recall with Manhatten-Distance:             95.20
2022-07-02 19:08:06 - Average Precision with Manhatten-Distance:  81.27

2022-07-02 19:08:06 - Accuracy with Euclidean-Distance:           73.87	(Threshold: 23.082

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

2022-07-02 19:08:11 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 14 after 10 steps:
2022-07-02 19:08:12 - Accuracy with Cosine-Similarity:           73.87	(Threshold: 0.7225)
2022-07-02 19:08:12 - F1 with Cosine-Similarity:                 81.94	(Threshold: 0.7225)
2022-07-02 19:08:12 - Precision with Cosine-Similarity:          72.39
2022-07-02 19:08:12 - Recall with Cosine-Similarity:             94.40
2022-07-02 19:08:12 - Average Precision with Cosine-Similarity:  81.06

2022-07-02 19:08:12 - Accuracy with Manhatten-Distance:           73.87	(Threshold: 575.3561)
2022-07-02 19:08:12 - F1 with Manhatten-Distance:                 82.07	(Threshold: 575.3561)
2022-07-02 19:08:12 - Precision with Manhatten-Distance:          72.12
2022-07-02 19:08:12 - Recall with Manhatten-Distance:             95.20
2022-07-02 19:08:12 - Average Precision with Manhatten-Distance:  81.29

2022-07-02 19:08:12 - Accuracy with Euclidean-Distance:           73.87	(Threshold: 23.044

### Using the auto-generated conclusions:

In [14]:
taska_training_df = pd.read_pickle('../data/TaskA_train_with_extra_conclusions.pkl')
taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')

In [15]:
#Mapping labels
taska_training_df = taska_training_df[taska_training_df.Validity != 0]
taska_valid_df    = taska_valid_df[taska_valid_df.Validity != 0]

taska_training_df['label'] = taska_training_df.Validity.apply(lambda x : 1 if x == 1 else 0)
taska_valid_df['label'] = taska_valid_df.Validity.apply(lambda x : 1 if x == 1 else 0)

taska_training_df['Premise'] = taska_training_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)
taska_valid_df['Premise'] = taska_valid_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)

In [16]:
_, _ = sbert_training.train_model(taska_training_df, taska_valid_df, 
                                output_path + '/task-A/validity/sbert/', 
                                'sentence-transformers/nli-roberta-large', 
                                num_epochs=15, train_batch_size=32,
                                model_suffix='extra-conclusions', max_seq_length=512, special_tokens=[], 
                                loss='ContrastiveLoss', sentence_transformer=False, evaluation_steps=10)

2022-07-02 19:09:16 - Use pytorch device: cuda
2022-07-02 19:09:16 - Read Triplet train dataset
Len of training: 4471
Len of Dev: 199
Evaluating before start learning.....
2022-07-02 19:09:16 - Binary Accuracy Evaluation of the model on sts-dev dataset:
2022-07-02 19:09:17 - Accuracy with Cosine-Similarity:           72.36	(Threshold: 0.6503)
2022-07-02 19:09:17 - F1 with Cosine-Similarity:                 80.69	(Threshold: 0.5985)
2022-07-02 19:09:17 - Precision with Cosine-Similarity:          70.91
2022-07-02 19:09:17 - Recall with Cosine-Similarity:             93.60
2022-07-02 19:09:17 - Average Precision with Cosine-Similarity:  82.54

2022-07-02 19:09:17 - Accuracy with Manhatten-Distance:           72.36	(Threshold: 646.3732)
2022-07-02 19:09:17 - F1 with Manhatten-Distance:                 80.56	(Threshold: 679.7071)
2022-07-02 19:09:17 - Precision with Manhatten-Distance:          71.17
2022-07-02 19:09:17 - Recall with Manhatten-Distance:             92.80
2022-07-02 19:09:1

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

2022-07-02 19:09:19 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 0 after 10 steps:
2022-07-02 19:09:19 - Accuracy with Cosine-Similarity:           72.36	(Threshold: 0.6541)
2022-07-02 19:09:19 - F1 with Cosine-Similarity:                 80.41	(Threshold: 0.5983)
2022-07-02 19:09:19 - Precision with Cosine-Similarity:          70.48
2022-07-02 19:09:19 - Recall with Cosine-Similarity:             93.60
2022-07-02 19:09:19 - Average Precision with Cosine-Similarity:  82.52

2022-07-02 19:09:19 - Accuracy with Manhatten-Distance:           72.36	(Threshold: 644.4963)
2022-07-02 19:09:19 - F1 with Manhatten-Distance:                 80.56	(Threshold: 678.1983)
2022-07-02 19:09:19 - Precision with Manhatten-Distance:          71.17
2022-07-02 19:09:19 - Recall with Manhatten-Distance:             92.80
2022-07-02 19:09:19 - Average Precision with Manhatten-Distance:  82.46

2022-07-02 19:09:19 - Accuracy with Euclidean-Distance:           72.86	(Threshold: 26.2884

...

2022-07-02 19:15:53 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 12 after 10 steps:
2022-07-02 19:15:54 - Accuracy with Cosine-Similarity:           62.31	(Threshold: -0.0416)
2022-07-02 19:15:54 - F1 with Cosine-Similarity:                 76.78	(Threshold: -0.1222)
2022-07-02 19:15:54 - Precision with Cosine-Similarity:          62.63
2022-07-02 19:15:54 - Recall with Cosine-Similarity:             99.20
2022-07-02 19:15:54 - Average Precision with Cosine-Similarity:  56.34

2022-07-02 19:15:54 - Accuracy with Manhatten-Distance:           63.82	(Threshold: 685.2755)
2022-07-02 19:15:54 - F1 with Manhatten-Distance:                 77.64	(Threshold: 685.2755)
2022-07-02 19:15:54 - Precision with Manhatten-Distance:          63.45
2022-07-02 19:15:54 - Recall with Manhatten-Distance:             100.00
2022-07-02 19:15:54 - Average Precision with Manhatten-Distance:  57.16

2022-07-02 19:15:54 - Accuracy with Euclidean-Distance:           62.31	(Threshold: 45.

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

2022-07-02 19:16:24 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 13 after 10 steps:
2022-07-02 19:16:24 - Accuracy with Cosine-Similarity:           62.81	(Threshold: -0.0296)
2022-07-02 19:16:24 - F1 with Cosine-Similarity:                 77.02	(Threshold: -0.0296)
2022-07-02 19:16:24 - Precision with Cosine-Similarity:          62.94
2022-07-02 19:16:24 - Recall with Cosine-Similarity:             99.20
2022-07-02 19:16:24 - Average Precision with Cosine-Similarity:  56.93

2022-07-02 19:16:24 - Accuracy with Manhatten-Distance:           63.82	(Threshold: 693.0916)
2022-07-02 19:16:24 - F1 with Manhatten-Distance:                 77.64	(Threshold: 693.0916)
2022-07-02 19:16:24 - Precision with Manhatten-Distance:          63.45
2022-07-02 19:16:24 - Recall with Manhatten-Distance:             100.00
2022-07-02 19:16:24 - Average Precision with Manhatten-Distance:  58.26

2022-07-02 19:16:24 - Accuracy with Euclidean-Distance:           62.81	(Threshold: 45.

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

2022-07-02 19:16:54 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 14 after 10 steps:
2022-07-02 19:16:54 - Accuracy with Cosine-Similarity:           62.81	(Threshold: -0.0502)
2022-07-02 19:16:54 - F1 with Cosine-Similarity:                 77.02	(Threshold: -0.0502)
2022-07-02 19:16:54 - Precision with Cosine-Similarity:          62.94
2022-07-02 19:16:54 - Recall with Cosine-Similarity:             99.20
2022-07-02 19:16:54 - Average Precision with Cosine-Similarity:  56.47

2022-07-02 19:16:54 - Accuracy with Manhatten-Distance:           63.82	(Threshold: 688.8479)
2022-07-02 19:16:54 - F1 with Manhatten-Distance:                 77.64	(Threshold: 688.8479)
2022-07-02 19:16:54 - Precision with Manhatten-Distance:          63.45
2022-07-02 19:16:54 - Recall with Manhatten-Distance:             100.00
2022-07-02 19:16:54 - Average Precision with Manhatten-Distance:  57.63

2022-07-02 19:16:54 - Accuracy with Euclidean-Distance:           62.81	(Threshold: 45.

#### Generating predictions for argument validity:

- Load best model and use it to predict the labels for the eval_df.
- Best model is: nli-roberta-large-ranking-loss-extra-data-2022-07-01_14-27-18

In [31]:
model_path = output_path + 'task-A/validity/sbert/sentence-transformers/nli-roberta-large-extra-conclusions-2022-07-02_19-09-07'
#model_path = output_path + 'task-A/validity/sbert/sentence-transformers/nli-roberta-large--2022-07-02_19-06-14'

In [32]:
taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')
taska_valid_df = taska_valid_df[taska_valid_df.Validity != 0]
taska_valid_df['Premise'] = taska_valid_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)

In [33]:
def predict_labels(df, model_path, clm1, clm2, threshold):
    best_model = SentenceTransformer(model_path)

    encoded_premises = best_model.encode(taska_valid_df[clm1].tolist())
    encoded_conclusions = best_model.encode(taska_valid_df[clm2].tolist())
    
    scores = [util.pytorch_cos_sim(x[0], x[1]).item() for x in zip(encoded_conclusions, encoded_premises)]
    labels = [1 if x > threshold else -1 for x in scores]
    
    df['pred_labels'] = labels
    
    return df

In [34]:
eval_df = predict_labels(taska_valid_df, model_path, 'Premise', 'Conclusion', 0.6146)
#eval_df = predict_labels(taska_valid_df, '../data/output/sentence-transformers/nli-roberta-large-ranking-loss-2022-07-01_14-24-04', 'Premise', 'Conclusion', 0.6146)

2022-07-02 19:21:34 - Load pretrained SentenceTransformer: ../../data-ceph/arguana/argmining22-sharedtask/models/task-A/validity/sbert/sentence-transformers/nli-roberta-large-extra-conclusions-2022-07-02_19-09-07
2022-07-02 19:21:36 - Use pytorch device: cuda


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [35]:
precision, recall, f1, _ = precision_recall_fscore_support(eval_df.Validity.tolist(), eval_df.pred_labels.tolist(), average='binary')

print('Precision: {}, Recall {}, F1: {}'.format(precision, recall, f1))

Precision: 0.70625, Recall 0.904, F1: 0.7929824561403509
